<a href="https://colab.research.google.com/github/harshyadav1508/GAN/blob/main/GanImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraray

In [1]:
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Input, Reshape, BatchNormalization, LeakyReLU
from sklearn import preprocessing
from keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Constants

In [2]:
noise_shape=(100,)
img_rows=28
img_cols=28
channel=1
batch_size=128
save_interval=50
img_shape=(img_rows, img_cols,channel)

## Loading Datasets

In [3]:
def load_data():
    (x_train, _), (_, _) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    # convert shape of x_train from (60000, 28, 28) to (60000, 784) 
    # 784 columns per row
    x_train = x_train.reshape(60000, img_rows* img_cols)
    return (x_train)

X_train=load_data()
print(X_train.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 784)


## Generator Function

In [4]:
def create_generator():

  model=Sequential()

  model.add(Dense(256, input_shape=noise_shape, activation=LeakyReLU(alpha=0.2)))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
  model.add(BatchNormalization(momentum=0.8))
  
  model.add(Dense(1024,  activation=LeakyReLU(alpha=0.2)))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
      

  model.compile(loss='binary_crossentropy', optimizer='Adam')
  return model

g=create_generator()
g.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4

## Discriminator Function

In [5]:
def create_discriminator():

  model=Sequential([
      Dense(1024,input_dim=np.prod(img_shape),activation=LeakyReLU(alpha=0.2)),
      Dense(512,activation=LeakyReLU(alpha=0.2)),
      Dense(256,activation=LeakyReLU(alpha=0.2)),
      Dense(1,activation='sigmoid'),
  ])
    
  model.compile(loss='binary_crossentropy', optimizer='Adam')
  return model

d =create_discriminator()
d.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              803840    
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,460,225
Trainable params: 1,460,225
Non-trainable params: 0
_________________________________________________________________


In [6]:
def create_gan(discriminator, generator):
    discriminator.trainable=False

    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)

    gan= Model(inputs=gan_input, outputs=gan_output)
    
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

gan = create_gan(d,g)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1493520   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1460225   
                                                                 
Total params: 2,953,745
Trainable params: 1,489,936
Non-trainable params: 1,463,809
_________________________________________________________________


## Saving Image

In [7]:
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/ML_project/GAN_MNIST/images/gan_generated_image %d.png' %epoch)

## Trainging GAN

In [ ]:
def training(epochs=1000, batch_size=128):
    
    #Loading the data
    (X_train) = load_data()
    batch_count =int( X_train.shape[0] / batch_size)
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_count)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 100])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=1
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
           plot_generated_images(e, generator)

training(400,128)

Epoch 1


100%|██████████| 468/468 [00:33<00:00, 13.81it/s]


Epoch 2


100%|██████████| 468/468 [00:28<00:00, 16.26it/s]


Epoch 3


...

100%|██████████| 468/468 [00:29<00:00, 15.71it/s]


Epoch 165


100%|██████████| 468/468 [00:29<00:00, 16.12it/s]


Epoch 166


 51%|█████▏    | 240/468 [00:15<00:15, 14.63it/s]